In [4]:
import requests
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup

In [5]:
url = "https://inshorts.com/sitemap-en.xml"
data = requests.get(url)
page = requests.get(url)
tree = html.fromstring(page.content)

In [6]:
count = len(tree.xpath("//url"))
raw_urls = []

In [7]:
str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","")

'https://inshorts.com/en/news/google-search-is-best-apple-ceo-on-billiondollar-deal-with-google-1542624460675'

In [8]:
for i in range(1,count+1):
    raw_urls.append(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']",""))

In [9]:
k = []
v = []
for i in range(1,count+1):
    #l = str(tree.xpath("//url["+i+"]/loc/text()")).replace("['","").replace("']","").count("-")
    #str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","").split("-")[:l]
    k.append(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']","").split("-")[-1])
    v.append(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']",""))

In [10]:
dict_url = {k[i]:v[i] for i in range(count)}

In [25]:
dict_url.keys

<function dict.keys>

In [30]:
#dict_url_pandas = pd.DataFrame.from_dict(dict_url, orient='index')
dict_url_pandas = pd.DataFrame(list(dict_url.items()), columns=['news_id','news_url'])

In [31]:
dict_url_pandas.head()

,news_id,news_url
0,1542337206761,https://inshorts.com/en/news/man-drives-burnin...
1,1542166787875,https://inshorts.com/en/news/traffic-policeman...
2,1542110804019,https://inshorts.com/en/news/amazon-wholesales...
3,1542125569107,https://inshorts.com/en/news/rename-andaman-ni...
4,1542099045961,https://inshorts.com/en/news/win-margin-was-sa...


l = str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","").count("-")
''.join(str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","").split("-")[:l]-
#str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","").split("-")[-1]

list(range(1,6))

raw_urls[1:3]

In [ ]:
pd.